In [1]:
import numpy as np

# finds the intersection point between the quadric 
# <x|Qx> + <b|x> + c = 0 (Q symmetric) and the line t -> a + tv
# i.e finds t such that 
# <a+tv|Q(a+tv)> + <b|a+tv> + c = 0
# <=> <a|Qa> + 2t<a|Qv> + t^2 <v|Qv> + <b|a> + t<b|v> + c = 0
# <=> A t^2 + B t + C = 0
# with A = <v|Qv>, B = 2<a|Qv> + <b|v>, C = c + <b|a> + <a|Qa> 

def ray_quadric_intersection(Q, b, c, a, v):
    A = np.dot(v,Q@v)
    B = 2*np.dot(a,Q@v) + np.dot(b,v)
    C = c + np.dot(b,a) + np.dot(a,Q@a)
    
    # linear case
    if np.abs(A) < 1e-7:
        print("linear case")
        if np.abs(B) > 1e-7:
            return -C/B
        else:
            return
        
    Delta = B**2 - 4*A*C
    if Delta <= 0:
        print("negative Delta={}".format(Delta))
        print("a={}, v={}".format(a,v))
        return
    sqrtDelta = np.sqrt(Delta)
    return (-B+sqrtDelta)/(2*A), (-B-sqrtDelta)/(2*A)

def segment_quadric_intersection(Q, b, c, p, q):
    T = ray_quadric_intersection(Q, b, c, p, q-p)
    T = [t for t in T if (t >= 0) and t <= 1]
    return T

def intermediate_point_on_quadric(Q, b, c, p, q, s, norm):
    v = np.cross(q-p, norm)
    #if v[2] > 0:
    #    v = -v
    a = (1-s)*p+s*q
    T = ray_quadric_intersection(Q, b, c, a, v)
    T = [t for t in T if t >= 0 and (a+t*v)[2] >=0]
    print(len(T))
    assert(len(T) == 1)
    return a + T[0]*v
    
def trace_curve_on_quadric(Q,b,c,p,q,norm,N):
    S = np.linspace(0,1,N)[1:-1]
    return [p]+[intermediate_point_on_quadric(Q,b,c,p,q,s,norm) for s in S]+[q]

In [2]:
# quadric parametrized by X -> sqrt(|X-p|^2 + beta) + b_i
# <=> z = sqrt((x-px)^2 + (y-py)^2 + beta) + b_i
# <=> (z-b_i)^2 = (x-px)^2 + (y-py)^2 + beta
# <=> <Q(X-R)|(X-R)> + beta = 0
# with Q = diag(1,1,-1), R = (px,py,bi)
# <=> <QX|X> + 2<QR|X> + <QR|R> + beta
Q = np.diag([1,1,-1])
R = np.zeros(3)
beta = 1.
b = 2*Q@R
c = np.dot(Q@R,R) + beta

def quad(x):
    return np.dot(x,Q@x) + np.dot(b,x) + c

# test 0
a = np.array([0,0,3])
v = np.random.rand(3)
T = ray_quadric_intersection(Q,b,c,a,v)
for t in T:
    print(quad(a+t*v))
    
# test 1
p = np.array([0,0,3])
q = np.random.rand(3)
q[2] = 0
T = segment_quadric_intersection(Q,b,c,p,q)
for t in T:
    print(quad(p+t*(q-p)))

# test 2
q = np.array([1.,0,0])
p = np.array([0.,1,0])
q[2] = np.sqrt(q[0]**2 + q[1]**2 + beta)
p[2] = np.sqrt(p[0]**2 + p[1]**2 + beta)
print("quad(p) = {}, quad(q) = {}".format(quad(p),quad(q)))

#norm = np.random.rand(3)
norm = np.array([0,0,-1])
# on veut p-q \in norm^\perp
# soit <p-q|norm> = 0
norm = norm - np.dot(p-q,norm)*(p-q)/np.linalg.norm(p-q)**2
print("n:", np.dot(p-q,norm))
print(norm)

curve = trace_curve_on_quadric(Q,b,c,p,q,norm,10)
for x in curve:
    print(x)
    print("{} {} {}".format(quad(x),np.dot(x-p,norm),np.dot(x-q,norm)))

    
import matplotlib.pyplot as plt
curve = np.array(curve)
plt.axis('equal')
plt.plot(curve[:,0], curve[:,1])

-3.552713678800501e-15
4.440892098500626e-16
-1.1102230246251565e-15
quad(p) = -4.440892098500626e-16, quad(q) = -4.440892098500626e-16
n: 0.0
[ 0.  0. -1.]
1
1
1
1
1
1
1
1
[0.         1.         1.41421356]
-4.440892098500626e-16 0.0 0.0
[0.20167477 0.97945255 1.41421356]
0.0 2.220446049250313e-16 2.220446049250313e-16
[0.37248333 0.92803888 1.41421356]
0.0 0.0 0.0
[0.5205176  0.85385094 1.41421356]
2.220446049250313e-16 -2.220446049250313e-16 -2.220446049250313e-16
[0.64936542 0.76047653 1.41421356]
0.0 0.0 0.0
[0.76047653 0.64936542 1.41421356]
0.0 0.0 0.0
[0.85385094 0.5205176  1.41421356]
-2.220446049250313e-16 0.0 0.0
[0.92803888 0.37248333 1.41421356]
2.220446049250313e-16 0.0 0.0
[0.97945255 0.20167477 1.41421356]
0.0 2.220446049250313e-16 2.220446049250313e-16
[1.         0.         1.41421356]
-4.440892098500626e-16 0.0 0.0


In [3]:
import pypower

In [4]:
X = np.random.rand(10,3)
w = np.zeros(10)
cells = pypower.power_diagram(X,w)

In [5]:
print(cells[0])

{-5: [array([1.        , 0.17643672, 0.36923051]), array([ 1.        , -0.21377572,  1.        ]), array([ 1., -1.,  1.]), array([ 1.        , -1.        , -0.04605871])], -4: [array([ 0.74682597, -1.        , -0.12663388]), array([ 1.        , -1.        , -0.04605871]), array([ 1., -1.,  1.]), array([ 0.55235312, -1.        ,  1.        ]), array([ 0.37737504, -1.        ,  0.59930719])], -1: [array([ 1., -1.,  1.]), array([ 1.        , -0.21377572,  1.        ]), array([ 0.61999755, -0.20033727,  1.        ]), array([ 0.55235312, -1.        ,  1.        ])], 4: [array([0.66233577, 0.24080269, 0.28448707]), array([1.        , 0.17643672, 0.36923051]), array([ 1.        , -1.        , -0.04605871]), array([ 0.74682597, -1.        , -0.12663388])], 5: [array([0.66233577, 0.24080269, 0.28448707]), array([0.47317778, 0.04186996, 0.61687055]), array([ 0.61999755, -0.20033727,  1.        ]), array([ 1.        , -0.21377572,  1.        ]), array([1.        , 0.17643672, 0.36923051])], 7: [a

In [8]:
import mpl_toolkits.mplot3d as a3
import matplotlib.colors as colors
import pylab as pl
import scipy as sp

ax = a3.Axes3D(pl.figure())
for c in cells:
    for k,vtx in c.items():
        print(k)
        #tri = a3.art3d.Poly3DCollection([vtx])
        #tri.set_color(colors.rgb2hex(sp.rand(3)))
        #tri.set_alpha(0.1)
        #tri.set_edgecolor('k')
        #ax.add_collection3d(tri)
        vtx = np.array(vtx)
        ax.plot3D(vtx[:,0], vtx[:,1], vtx[:,2])
ax.autoscale()

-5
-4
-1
4
5
7
9
-5
-3
-2
2
4
6
8
-6
-3
-2
1
3
4
5
6
7
8
-6
-3
-1
2
5
6
7
9
-5
-4
-2
0
1
2
5
7
8
-5
-1
0
2
3
4
6
7
8
9
-3
-1
1
2
3
5
7
8
9
-6
-4
-2
0
2
3
4
5
6
9
-5
-3
-1
1
2
4
5
6
-6
-4
-1
0
3
5
6
7


In [7]:
%matplotlib qt